In [8]:
import pandas as pd


In [9]:
#read in the markdown file at D:\repos\project_amanita\data\anon_pdf.md
#add folder to python path

file_path = r"..\data\anon_pdf.md"
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()
    f.close()

In [10]:
import re
from typing import List, Dict
from urllib.parse import urlparse

def parse_ufo_data(text: str) -> List[Dict]:
    data = []
    event_blocks = text.split('**(PUBLIC DOMAIN)** - ')

    for event_block in event_blocks[1:]:
        event = {}
        lines = event_block.split('\n')
        
        date_match = re.search(r'\*\*([0-9]{1,2} [A-Za-z]+ [0-9]{4}|[A-Za-z]+ [0-9]{4}|[0-9]{4})\*\*', lines[0])
        event['date'] = date_match.group(1) if date_match else None

        desc_start = len(event['date']) + 6 if event['date'] else 0
        event['description'] = lines[0][desc_start:].strip()
        
        # Initialize sources as a list to handle multiple sources
        event['sources'] = []
        # Initialize comments as a list to handle multiple comments
        event['comments'] = []
        for line in lines[1:]:
            if line.startswith('* [http'):
                source_match = re.search(r'\[([^]]*)\]', line)
                source = source_match.group(1) if source_match else None
                netloc = urlparse(source).netloc if source else None
                source_link = re.search(r'\(([^)]+)\)', line).group(1)
                # Append each source as a dictionary to sources list
                event['sources'].append({"source": netloc, "source_link": source_link})
            elif line.startswith('>**_Note_**:'):
                # Extract comments and split by ':'
                comments_list = line[11:].split(':')
                # Strip each comment and append to comments list
                event['comments'] += [comment.strip() for comment in comments_list if comment.strip()]
                
        data.append(event)
        
    return data


In [11]:

#lets turn this data into a dataframe
import pandas as pd
df = pd.DataFrame(parse_ufo_data(text))
df.head()


,date,description,sources,comments
0,8 July 1947,1st Lt. Walter Haut states in an allegedly sig...,"[{'source': 'www.the-sun.com', 'source_link': ...",[]
1,8 July 1947,"Another affidavit from Arthur R. McQuiddy, the...",[],[]
2,8 July 1947,Col. Thomas J. DuBose states in a signed affid...,[],[]
3,8 July 1947,A signed affidavit from 1st Lt. Robert Shirkey...,[],[]
4,19 August 1947,USAF Lt. Col. George Garrett tells FBI SAC S.W...,[],[]


In [12]:
df.to_csv('../data/parsed_data.csv', index=False)